In [1]:
# 执行本脚本更新flags列表文件： iwencai_flags.json / iwencai_flags_withtype.json
# 写入文件之前对flag进行标准化：
# 去除“停牌”，去除所有“最近”开头的flag
# 将“今日”开头的flag的“今日”两字去除

from opentrader.core.crawler import *
from opentrader.agents.xueqiu.api import *
from opentrader.agents.ths.api import *
c = CNCrawler()
stock_list = c.get_stock_list()

Querying 54 pages ......................................................

In [2]:
import json, threading, sys
from datetime import datetime,timedelta
stockflags = [] # [{'symbol':'', 'buy':[], 'sell':[], 'event':[], 'zxst':[]}, ...]
lock = threading.RLock()
ths = THSAPI()

# create 10 threads to do the job
class StockAnalyzer(threading.Thread):
    def __init__(self, stocks):
        threading.Thread.__init__(self)
        self.stocks = stocks
        
    def run(self):
        # for each stock, get latest flags
        for i,stock in enumerate(self.stocks):
            #if (i > 100):
            #    break
            print('Querying stock %d, %s...' % (i, stock['symbol']))
            sys.stdout.flush()
            try:
                result = ths._get_stock_analyze(stock['symbol'])
            except InvalidSymbol:
                continue
            stockflag = {'symbol':stock['symbol'], 'buy':[], 'sale':[], 'event':[], 'zxst':[]}
            for key in ('event','buy','sale','zxst'):
                #print(result['data'])
                try:
                    flag_dict = result['data']['data']['result'][key]
                except (TypeError, KeyError):
                    continue
                for (k, flag_item) in flag_dict.items():
                    flag = flag_item['query']
                    stockflag[key].append(flag)
            lock.acquire()
            stockflags.append(stockflag)
            lock.release()

each_len = len(stock_list) / 10
threads = []
for i in range(100):
    stocks = stock_list[i*400:(i+1)*400]
    if len(stocks) == 0:
        break
    thread = StockAnalyzer(stocks)
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()

f = open('flags' + str(datetime.now().date())+'.json', 'w')
f.write(json.dumps(stockflags))

Querying stock 0, HKHSF...
Querying stock 1, SH603999...
Querying stock 0, SH600797...
Querying stock 0, SH600322...
Querying stock 0, SZ399682...
Querying stock 0, SZ300377...
Querying stock 0, SZ200418...
Querying stock 0, SH104712...
Querying stock 0, SH104023...
Querying stock 0, SZ002772...
Querying stock 0, SZ002367...
Querying stock 0, SZ000981...
Querying stock 0, SZ000601...
Querying stock 1, SZ200413...
Querying stock 1, SH600321...
Querying stock 1, SH104711...
Querying stock 1, SH600796...
Querying stock 1, SZ000980...
Querying stock 1, SH104022...
Querying stock 1, SZ399681...
Querying stock 2, SH603998...
Querying stock 1, SZ002771...
Querying stock 1, SZ002366...
Querying stock 1, SZ300376...
Querying stock 1, SZ000600...
Querying stock 2, SZ200168...
Querying stock 2, SH600320...
Querying stock 2, SH000980...
Querying stock 2, SZ002770...
Querying stock 2, SH104021...
Querying stock 3, SZ200160...
Querying stock 2, SZ000599...
Querying stock 3, SH600319...
Querying stoc

3441429

In [3]:
print(len(stock_list))
print(len(stockflags))

4728
4727


In [19]:
# make statistics over all flags and update to file "iwencai_flags.json"
try:
    f = open('iwencai_flags.json', 'r')
    flags = json.loads(f.read())
    print('total flags: %d' % len(flags))
    f.close()
except:
    flags = []

flags_withtype = {'buy':[], 'sale':[], 'event':[], 'zxst':[]}
for each in stockflags:
    for key in ('buy', 'sale', 'event', 'zxst'):
        for each_flag in each[key]:
            # 标准化flag
            excludes = ("停牌", "最近", "持续", "连阴", "连阳", "连涨", "连跌", "持股")
            exc = False
            for each_ex in excludes:
                if each_ex in each_flag:
                    exc = True
                    break
            if exc:
                continue
            if "上穿5日" in each_flag or "下穿5日" in each_flag:
                each_flag = each_flag + "均线"
            if each_flag.startswith("今日"):
                each_flag = each_flag[2:]
            if each_flag not in flags:
                flags.append(each_flag)
                print('Added flag: %s' % each_flag)
            if each_flag not in flags_withtype[key]:
                flags_withtype[key].append(each_flag)
print('---------After Update----------')
print('total flags: %d' % len(flags))
f = open('iwencai_flags.json', 'w')
f.write(json.dumps(flags))
f.close()
f = open('iwencai_flags_withtype.json', 'w')
f.write(json.dumps(flags_withtype))
f.close()
print(flags_withtype)

Added flag: 月线mtm金叉
Added flag: 周线skdj金叉
Added flag: mtm金叉
Added flag: bias卖出信号
Added flag: 周线bias卖出信号
Added flag: w&r超买
Added flag: 周线w&r超买
Added flag: 秋影金波
Added flag: 周线价升量缩
Added flag: 月线低位大阳
Added flag: 周线缩量
Added flag: 周线底部缩量
Added flag: 四阳夹一阴
Added flag: 周线中阳线
Added flag: 月线连续拉升
Added flag: 月线尖三兵
Added flag: 月线价升量缩
Added flag: 月线大阳线
Added flag: 月线秋影金波
Added flag: 月线低开高走
Added flag: 周线阳线
Added flag: 月线底部吸筹
Added flag: 月线聚宝盆
Added flag: 周线反弹
Added flag: 月线阳线
Added flag: 放量
Added flag: v型反转
Added flag: 小三阳
Added flag: 价升量涨
Added flag: 阳线
Added flag: 月线博弈K线长阳
Added flag: 周线低开高走
Added flag: 月线阶段放量
Added flag: cci买入信号
Added flag: 周线行情收盘价上穿5日均线
Added flag: 月线rsi金叉
Added flag: dma金叉
Added flag: 月线bias卖出信号
Added flag: 月线上升通道
Added flag: 太阳线
Added flag: 周线旗形
Added flag: 月价托
Added flag: 周线秋影金波
Added flag: 月线长下影线
Added flag: kdj超卖
Added flag: 月线kdj超卖
Added flag: 周线skdj超卖
Added flag: 周线kdj超卖
Added flag: 股价创历史新高
Added flag: 涨停
Added flag: 连续3日涨停
Added flag: 周线博弈K线长阳
Added flag: 跳空高开
Added fla

In [10]:
# check today's stocks with most buy flags
import operator
from datetime import datetime,timedelta
f = open('flags' + str(datetime.now().date())+'.json', 'r')
#f = open('flags2015-12-21.json', 'r')
stockflags = json.loads(f.read())
f.close()

f = open('../system/iwencai_flag_performance_onedayf.json', 'r')
flagperformance = json.loads(f.read())
f.close()

stockmeans = {}
stock_maxflag = {}
totalflags = 0
totalmean = 0
stock_current = {}
for each in stock_list:
    stock_current[each['symbol']] = float(each['percent'])
print(stock_current['SH600469'])
for each in stockflags:
    stockmean = 0
    stock_flag_count = 0
    maxflag = ['', 0]
    for key in ('buy', 'sale', 'event', 'zxst'):
        for flag in each[key]:
            if flag == '停牌':
                continue
            if flag not in flagperformance:
                continue
            stock_flag_count += 1
            stockmean += flagperformance[flag]['avr']
            totalflags += 1
            totalmean += flagperformance[flag]['avr']
            if flagperformance[flag]['avr'] > maxflag[1]:
                maxflag[1] = flagperformance[flag]['avr']
                maxflag[0] = flag
    if stock_flag_count == 0:
        continue
    stockmean = stockmean / stock_flag_count
    #if stock_current[each['symbol']] > 9.8:
    #    continue
    stockmeans[each['symbol']] = stockmean
    stock_maxflag[each['symbol']] = maxflag
            
sorted_x = sorted(stockmeans.items(), key=operator.itemgetter(1))
sorted_x.reverse()
print(totalmean/totalflags/0.8)
print(len(sorted_x))
for each in sorted_x[:100]:
    print("symbol: %s, mean: %f, max flag: %s, max flag mean: %f" % (each[0], each[1], stock_maxflag[each[0]][0], stock_maxflag[each[0]][1]))


-2.39
0.6535114258488003
2785
symbol: SZ002526, mean: 1.123937, max flag: 月线秋影金波, max flag mean: 2.723208
symbol: SH600469, mean: 1.106289, max flag: 月线秋影金波, max flag mean: 2.723208
symbol: SZ002782, mean: 1.065564, max flag: 周线光头光脚阳线, max flag mean: 5.567342
symbol: SZ002781, mean: 1.065564, max flag: 周线光头光脚阳线, max flag mean: 5.567342
symbol: SZ300495, mean: 1.065564, max flag: 周线光头光脚阳线, max flag mean: 5.567342
symbol: SZ300497, mean: 1.065564, max flag: 周线光头光脚阳线, max flag mean: 5.567342
symbol: SZ002001, mean: 1.046857, max flag: 月线秋影金波, max flag mean: 2.723208
symbol: SH600636, mean: 1.040983, max flag: 月线秋影金波, max flag mean: 2.723208
symbol: SZ000715, mean: 1.035564, max flag: 周线光头光脚阳线, max flag mean: 5.567342
symbol: SZ002132, mean: 1.023666, max flag: 周线光头光脚阳线, max flag mean: 5.567342
symbol: SH600790, mean: 1.013440, max flag: 月线秋影金波, max flag mean: 2.723208
symbol: SH600741, mean: 1.008801, max flag: 月线秋影金波, max flag mean: 2.723208
symbol: SH600830, mean: 1.005362, max flag: 月线